In [1]:
#@title Install libraries, imports and variables
#@markdown Please "Connect with Drive" before proceeding
#@markdown 
#@markdown Run only once

import os


if os.path.isdir(f'./drive/'):
  %tensorflow_version 1.x
  !pip install -q gpt-2-simple

  print('Libraries installed...')

  #import tensorflow
  import gpt_2_simple as gpt2
  from datetime import datetime
  import numpy as np
  import tensorflow as tf
  import tensorflow_hub as hub
  import gc

  print('Imports finished...')

  #tf.enable_eager_execution()
  #tf.executing_eagerly()

  formal_path = "/content/drive/My Drive/Colab Notebooks/Corpus/model_frml_USE"

else:
  print("Please Connect with Drive")
  from google.colab import drive
  drive.mount('/content/drive')

#@markdown ____


TensorFlow 1.x selected.
Libraries installed...
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Imports finished...


In [2]:
#@title Select run { run: "auto" }
run_name = "run1" #@param {type:"string"}

if not os.path.isdir(f'./checkpoint/{run_name}'):
  print(f"Copy checkpoint {run_name}...")
  gpt2.copy_checkpoint_from_gdrive(run_name=run_name)
else:
  print(f"Checkpoint {run_name} already exist")


print('Starting TF sess...')
tf.reset_default_graph()
sess = gpt2.start_tf_sess(threads=-1)
gpt2.load_gpt2(sess, run_name=run_name)
print(f'TF sess {run_name} loaded')

sess2 = tf.Session()

with sess2.as_default():
  embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")

sess2.run([tf.global_variables_initializer(), tf.tables_initializer()])

def get_meaning_pres(a,b):
  #sess2.run([tf.global_variables_initializer(), tf.tables_initializer()])
  embeddings = embed(
        [a] + b
    )
    
  message_embeddings = sess2.run(embeddings)

  return np.inner(message_embeddings[0],message_embeddings[1:])

#@markdown ____


Checkpoint run1 already exist
Starting TF sess...
Loading checkpoint checkpoint/run1/model-2000
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-2000
TF sess run1 loaded
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [3]:
#@title Set variables { run: "auto" }

#@markdown ```informal_input: max 300 characters```
informal_input = "im here but i dunno why" #@param {type:"string"}
if len(informal_input) > 300:
  print('Input too long, triming...')
  informal_input = informal_input[:300]

#@markdown ```samples: range [1,10]```
samples = 7 #@param {type:"slider", min:0, max:10, step:1} 


#@markdown ```temperature: range [0,1]```
temperature = 0.7 #@param {type:"slider", min:0, max:1, step:0.05} 

#@markdown ```get_metrics: formality and meaning preservation```
get_metrics = True #@param {type:"boolean"}

print(f"Input:", informal_input)
print(f"Samples:", samples)
print(f"Temperature:", temperature)

#@markdown ____



Input: im here but i dunno why
Samples: 7
Temperature: 0.7


In [4]:
#@title Run model 
#@markdown Run this cell to execute the transfer. It will take up to a minute the first time.
#@markdown ____
print('Generando...')
arr = gpt2.generate(sess,
            run_name = run_name,
            length=300,
            temperature=temperature,
            prefix=f"<|startoftext|>[Informal]{informal_input}\n[Formal]",
            truncate = '<|endoftext|>',
            nsamples=samples,
            batch_size=samples,
            include_prefix=False,
            return_as_list=True)
arr = list(set(arr))
arr = [x.replace("[Formal]","").replace("[Informal]","").replace("<|startoftext|>","") for x in arr]

if get_metrics:
  mean_press = get_meaning_pres(informal_input,arr)

print('Generado...\n')

print(f"Input sentence: {informal_input}")
print('Outputs'.ljust(100),f'{"Meaning Preservation" if get_metrics else ""}'.ljust(10))
for i in range(len(arr)):
  print(arr[i].ljust(100),str(mean_press[i]).ljust(10))

if get_metrics:
  print('\n\n\nCopy the following line to Formality Metric Notebook:')
  print(arr)

Generando...
Generado...

Input sentence: im here but i dunno why
Outputs                                                                                              Meaning Preservation
I am here, but I do not know why.                                                                    0.9121772 
I am here, however I do not know why.                                                                0.8429709 
I'm here, but I'm unsure why.                                                                        0.9717683 



Copy the following line to Formality Metric Notebook:
['I am here, but I do not know why.', 'I am here, however I do not know why.', "I'm here, but I'm unsure why."]
